<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm import notebook
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [2]:
data = pd.read_csv('toxic_comments.csv')
data

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


В данных нет пропусков тип данных соответсвует содержимому. Столбец `Unnamed:0` кажется бесполезным. Он не пригодится в дальнейшем обучении.

Приступим к обучению

## Обучение

Проведем токенизацию комментариев. Для этого воспользуемся `BertTokenizer`. В качестве самой модели используем предобученную `unitary/toxic-bert` которая предназначена для токсичных комментариев на английском языке  

Сохраним `BertTokenizer` в переменную `tokenizer`

In [4]:
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')

Применим токенайзер для наших данных и сохраним их в переменную `tokenized`

Установим дополнительные параметры для выделения начала и конца, а так же чтобы максимальная длина не привышала 512 т.к. BERT работаем не более чем с 512 признаками

In [5]:
tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, padding=True))

Посчитаем самую длинную строку и сохраним ее в переменную `max_len`

In [6]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

512


Заполним остальные строки нулями до размеров максимально длинной строки и сохраним в переменную `padded`

In [7]:
padded = np.array(list(map(lambda i: i + [0]*(max_len - len(i)), tokenized.values)))

Так же для BERT создадим маску в которой важные для обучения значения указываются как 1 а бесполезные заполненные для размерности 0. Сохраним маску в переменную `attention_mask`

In [8]:
attention_mask = np.where(padded != 0, 1, 0)

Сохраним конфигурацию в переменной `config`

In [9]:
config = BertConfig.from_pretrained('unitary/toxic-bert')

И саму модель с конфигом в переменную  `model`

In [10]:
model = BertModel.from_pretrained('unitary/toxic-bert', config=config)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Чтобы эмбеддинг не занимал бесконечно много времени с нашим датасетом почти в 160 тыс. строк, возьмем случайные 500 строк из наших переменных: 
- `padded`
- `attention_mask`
- `data['toxic']` (наш целевой признак target)

In [11]:
padded_500, padded_else, attention_mask_500, attention_mask_else, target_500, target_else = train_test_split(padded, attention_mask, data['toxic'], train_size=500, random_state=333)

print(padded_500.shape, attention_mask_500.shape, target_500.shape)

(500, 512) (500, 512) (500,)


In [12]:
padded = padded_500
attention_mask = attention_mask_500

Приступим к процессу эмбеддинга. Чтобы не возникло проблем с нехватков оперативной памяти разделим этот процесс на части.

Создадим переменные:
- `batch_size` (для разбиения данных на этапы) 
- `embeddings` (список для сохранения результатов эмбеддинга)

In [13]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
  
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/5 [00:00<?, ?it/s]

Сохраним результаты:
- эмбеддинга в `features`
- срез целевого признака в `target`

In [14]:
features = np.concatenate(embeddings)
target = target_500

Разделим на обучающую и тестовую выборки указав **stratify=target** чтобы и в тестовую и обучающу выборку попали одинаковое количество классов

In [15]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.5, random_state=333, stratify=target)

Обучим модель логистической регрессии на эмбеддингах. Подберем параметр регуляризации с помощью гридсерч.

In [16]:
model = LogisticRegression(random_state=333)

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

In [17]:
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='f1')

In [18]:
grid_search.fit(train_features, train_target)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=333),
             param_grid={'C': [0.01, 0.1, 1, 10, 100]}, scoring='f1')

In [19]:
print(f'Параметр регуляризации: {grid_search.best_params_}')
print(f'F1-score: {grid_search.best_score_:.2f}')

Параметр регуляризации: {'C': 0.01}
F1-score: 0.94


При кросс валидации нашли лучший параметр регулязирации в `0.01` с лучшим средним показателем метрики `F1`на обучающей выборке в `0.94`

Логистическая регрессия обученная на эмбеддингах дала отличный результат. Не вижу смысла дальнейшего обучения других моделей

Обучим нашу модель с подобранным параметром и проведем итоговое тестирование

In [20]:
model = LogisticRegression(C = 0.01, random_state=333)

In [21]:
model.fit(train_features, train_target)
predict = model.predict(test_features)

Проверим Значения метрик **accuracy** и **F1-score**

In [22]:
print('accuracy на тестовой выборке: ', accuracy_score(test_target, predict))
print('F1-score на тестовой выборке: ', f1_score(test_target, predict))

accuracy на тестовой выборке:  0.98
F1-score на тестовой выборке:  0.8936170212765957


## Выводы

Получили эмбеддинги с помощью нейронной сети BERT для 500 случайных строк из исходных данных. Обучение проводилось на 250 строках. 

При итогововм тестировании модели Логистической регрессии обученой на эмбеддингах полученных с помощью BERT на тестовой выборке (размером в 250 строк) получили результаты метрик:
- `accuracy: 0.98`
- `F1-score: 0.89`